<a href="https://colab.research.google.com/github/jeanlucjackson/w266_final_project/blob/main/code/BB/bb_t5_simple_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Transformers Attempt

### Imports and installs

In [1]:
import numpy as np
import pandas as pd

import json

# Make longer output readable without scrolling
from pprint import pprint

# Stop warning messages from showing
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
!pip install -q datasets

     |████████████████████████████████| 441 kB 22.8 MB/s 
     |████████████████████████████████| 163 kB 65.2 MB/s 
     |████████████████████████████████| 212 kB 64.7 MB/s 
     |████████████████████████████████| 115 kB 73.0 MB/s 
     |████████████████████████████████| 127 kB 71.0 MB/s 
     |████████████████████████████████| 115 kB 71.1 MB/s 


In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


### Prep data

In [4]:
from datasets import list_datasets, load_dataset_builder, get_dataset_config_names, load_dataset, load_from_disk

In [5]:
def summarize_dataset (dataset, config=None):
    builder = load_dataset_builder(dataset, config)
    print(f"Description:\n {builder.info.description}")
    print(f"Features:")
    pprint(builder.info.features)
    return

In [6]:
data_squad = load_from_disk("/content/drive/MyDrive/w266 NLP Final Project/Data/squad.hf")

In [7]:
data_squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

Set up dataframes

In [8]:
# Create dataframe for train data 
# shuffle so random
count=10000
data_shuffle=data_squad['train'].shuffle(seed=1962).select(range(count))
train_df=pd.DataFrame()
train_df['answer'] = [answer['text'][0] for answer in data_shuffle['answers']]
train_df['question'] = data_shuffle['question']
train_df['description'] = data_shuffle['context']
train_df['txt'] = [f"answer: {answer} context: {context}" for answer, context in zip (train_df.answer, train_df.description)]
train_df['prefix'] = 'ask_question'
train_df = train_df[['question', 'txt', 'prefix']]
train_df.columns = ['target_text', 'input_text', 'prefix']


train_df.head()

,target_text,input_text,prefix
0,What type of businesses did Nickles want to at...,answer: biotech companies context: Prior to mo...,ask_question
1,To whom did Chopin reveal in letters which par...,answer: Tytus Woyciechowski context: Four boar...,ask_question
2,"If a species may be harmed, who holds final sa...",answer: the Endangered Species Committee conte...,ask_question
3,What country has the dog as part of its 12 ani...,answer: China context: In Asian countries such...,ask_question
4,How long did his episcopate last?,answer: 45 years context: Saint Athanasius of ...,ask_question


In [10]:
# Create dataframe for validation data 
# shuffle so random
count=1000
data_shuffle=data_squad['validation'].shuffle(seed=1962).select(range(count))
eval_df=pd.DataFrame()
eval_df['answer'] = [answer['text'][0] for answer in data_shuffle['answers']]
eval_df['question'] = data_shuffle['question']
eval_df['description'] = data_shuffle['context']
eval_df['txt'] = [f"answer: {answer} context: {context}" for answer, context in zip (eval_df.answer, eval_df.description)]
eval_df['prefix'] = 'ask_question'
eval_df = eval_df[['question', 'txt', 'prefix']]
eval_df.columns = ['target_text', 'input_text', 'prefix']


eval_df.head()

,target_text,input_text,prefix
0,How many levels of galleries do the façades su...,answer: four context: Prince Albert appears wi...,ask_question
1,What are the secretions commonly called?,"answer: ink context: When some species, includ...",ask_question
2,When did Newcastle's first indoor market open?,answer: 1835 context: The Grainger Market repl...,ask_question
3,What may be presented to Parliament in various...,answer: Bills context: Bills can be introduced...,ask_question
4,"Prior to the arrival of the French, the area n...",answer: the Timucua context: Jacksonville is i...,ask_question


## Simple Transformers

Sources: 

https://towardsdatascience.com/asking-the-right-questions-training-a-t5-transformer-model-on-a-new-task-691ebba2d72c

https://simpletransformers.ai/docs/t5-specifics/

https://github.com/ThilinaRajapakse/simpletransformers

### Install Requirements

In [11]:
!pip install -q simpletransformers

     |████████████████████████████████| 250 kB 32.4 MB/s 
     |████████████████████████████████| 5.3 MB 65.2 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 1.3 MB 57.1 MB/s 
     |████████████████████████████████| 7.6 MB 62.4 MB/s 
     |████████████████████████████████| 9.2 MB 53.9 MB/s 
     |████████████████████████████████| 1.9 MB 68.0 MB/s 
     |████████████████████████████████| 182 kB 75.1 MB/s 
     |████████████████████████████████| 166 kB 73.5 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 166 kB 65.7 MB/s 
     |████████████████████████████████| 162 kB 78.9 MB/s 
     |████████████████████████████████| 162 kB 78.4 MB/s 
     |████████████████████████████████| 158 kB 78.5 MB/s 
     |████████████████████████████████| 157 kB 81.9 MB/s 
     |████████████████████████████████| 157 kB 72.3 MB/s 
     |████████████████████████████████| 157 kB 79.7 MB/s 
     |████████████

In [12]:
# !pip uninstall torch torchvision -y   # uncomment this line if having issues with cuda
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# This isn't needed unless using fp16 training

# !git clone https://github.com/NVIDIA/apex
# %cd apex
# !pip install -v --no-cache-dir ./

### Training

In [13]:
from simpletransformers.t5 import T5Model

In [14]:
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 128,
    "train_batch_size": 32,
    "num_train_epochs": 1,
    "save_eval_checkpoints": True,
    "save_steps": -1,
    "use_multiprocessing": False,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 1000,
    "evaluate_during_training_verbose": True,
    "fp16": False,

    "wandb_project": "Question Generation with T5",
}

model = T5Model("t5", "t5-base", args=model_args)

model.train_model(train_df, eval_data=eval_df)

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3543: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Running Epoch 0 of 1:   0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3543: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


(313,
 {'global_step': [313],
  'eval_loss': [1.8316443271636962],
  'train_loss': [1.900877594947815]})

Can see Weights and Biases at: https://wandb.ai/w266/Question%20Generation%20with%20T5?workspace=user-bronte-baer

*Note: entirely sure how best to use W&B yet...*

### Save and load trained model to drive

In [ ]:
# THIS IS NO LONGER NEEDED
# because I figured out how to access the outputs file for the model

# import torch


# # path = "/content/drive/MyDrive/w266 NLP Final Project/Checkpoints/bb_t5_st")

# # Specify a path
# path = "bb_t5_st.pt"

# # Save
# torch.save(model, path)

# # Load
# model = torch.load(path)

### Evaluation

Choose some text to test/evaluate

In [15]:
test_txt1 = "In Asian countries such as China, Korea, and Japan, dogs are viewed as kind protectors. The role of the dog in Chinese mythology includes a position as one of the twelve animals which cyclically represent years (the zodiacal dog)."

test_txt2 = "After Hurricane Katrina in 2005, Beyoncé and Rowland founded the Survivor Foundation to provide transitional housing for victims in the Houston area, to which Beyoncé contributed an initial $250,000. The foundation has since expanded to work with other charities in the city, and also provided relief following Hurricane Ike three years later."

# test_txt3 = ""


In [17]:
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 128,
    "eval_batch_size": 128,
    "num_train_epochs": 4,
    "save_eval_checkpoints": False,
    "use_multiprocessing": False,
    "num_beams": None,
    "do_sample": True,
    "max_length": 50,
    "top_k": 50,
    "top_p": 0.95,
    "num_return_sequences": 1,
}

model = T5Model("t5", "outputs", args=model_args)

query = "ask_question: " + test_txt1

preds = model.predict([query])

print(preds)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['What are the Chinese belief that dogs are kind?']


In [ ]:
help(model)

Help on T5Model in module simpletransformers.t5.t5_model object:

class T5Model(builtins.object)
 |  T5Model(model_type, model_name, args=None, tokenizer=None, use_cuda=True, cuda_device=-1, **kwargs)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, model_type, model_name, args=None, tokenizer=None, use_cuda=True, cuda_device=-1, **kwargs)
 |      Initializes a T5Model model.
 |      
 |      Args:
 |          model_type: The type of model (t5, mt5, byt5)
 |          model_name: The exact architecture and trained weights to use. This may be a Hugging Face Transformers compatible pre-trained model, a community model, or the path to a directory containing model files.
 |          args (optional): Default args will be used if this parameter is not provided. If provided, it should be a dict containing the args that should be changed in the default args.
 |          use_cuda (optional): Use GPU if available. Setting to False will force model to use CPU only.
 |          cuda_device (o